# 从神经元到神经网络

神经网络是多个“神经元”（感知机）的带权级联，神经网络算法可以提供非线性的复杂模型，它有两个参数：权值矩阵$W^l$和偏置向量$b^l$，不同于感知机的单一向量形式，$W^l$是复数个矩阵，$b^l$是复数个向量，其中的元素分别属于单个层，而每个层的组成单元，就是神经元。

## 神经元

神经网络是由多个“神经元”（感知机）组成的，每个神经元图示如下：

![](cnn0.jpg)

这其实就是一个单层感知机，其输入是由$x_1,x_2,x_3$和+1组成的向量，其输出为$h_{W,b}(x)=f(W^T x)=f(\sum_{i=1}^{3}W_i x_i+b)$，其中$f$是一个激活函数，模拟的是生物神经元在接受一定的刺激之后产生兴奋信号，否则刺激不够的话，神经元保持抑制状态这种现象。这种由一个阈值决定两个极端的函数有点像示性函数，然而这里采用的是Sigmoid函数，其优点是连续可导。

### Sigmoid函数

常用的Sigmoid有两种——

#### 单极性Sigmoid函数

$$f(x)=\frac{1}{1+e^{-x}}$$

其图像如下

![](cnn1.jpg)


#### 双极性Sigmoid函数

$$f(x)=\frac{e^{x}-e^{-x}}{e^{x}+e^{-x}}$$

或者写成

$$f(x)=\frac{1-e^{-x}}{1+e^{-x}}$$

把第一个式子分子分母同时除以$e^x$，令$x=-2x$就得到第二个式子了，换汤不换药。

其图像如下

![](cnn2.jpg)


从它们两个的值域来看，两者名称里的极性应该指的是正负号。从导数来看，它们的导数都非常便于计算：

对于$$f(x)=\frac{1}{1+e^{-x}}$$有$f'(x)=f(x)(1-f(x))$，对于tanh，有$f'(x)=1-(f(x))^2$。

> 1/(1+e^-x)求导的过程：
  $$
\begin{align}
\frac{d}{dx}\sigma&=\frac{d}{dx}(\frac{1}{1+e^{-x}})\\
&=\frac{e^{-x}}{(1+e^{-x})^2}\\
&=\frac{(1+e^{-x})-1}{(1+e^{-x})^2}\\
&=\frac{(1+e^{-x})}{(1+e^{-x})^2}-\frac{1}{(1+e^{-x})^2}\\
&=\sigma(x)-\sigma(x)^2\\
\sigma' &= \sigma(1-\sigma)
\end{align}
  $$
  一旦知道了$f(x)$，就可以直接求$f'(x)$，所以说很方便。

本Python实现使用的就是$\frac{1}{1+e^{-x}}$

In [ ]:
def sigmoid(x):
    """
    sigmoid 函数，1/(1+e^-x)
    :param x:
    :return:
    """
    return 1.0/(1.0+math.exp(-x))
 
def dsigmoid(y):
    """
    sigmoid 函数的导数
    :param y:
    :return:
    """
    return y * (1 - y)

也可以使用双曲正切函数tanh

In [ ]:
def sigmoid(x):
    """
    sigmoid 函数，tanh 
    :param x:
    :return:
    """
    return math.tanh(x)

def dsigmoid(y):
    """
    sigmoid 函数的导数
    :param y:
    :return:
    """
    return 1.0 - y ** 2

## 神经网络模型(BPNN)

神经网络就是多个神经元的级联，上一级神经元的输出是下一级神经元的输入，而且信号在两级的两个神经元之间传播的时候需要乘上这两个神经元对应的权值。例如，下图就是一个简单的神经网络：

![](cnn3.jpg)

其中，一共有一个输入层，一个隐藏层和一个输出层。输入层有3个输入节点，标注为+1的那个节点是偏置节点，偏置节点不接受输入，输出总是+1。

定义上标为层的标号，下标为节点的标号，则本神经网络模型的参数是：$(W,b)=(W^{(1)},b^{(1)},W^{(2)},b^{(2)})$，其中$W_{ij}^{(l)}$是第$l$层的第$j$个节点与第$l+1$层第$i$个节点之间的连接参数（或称权值）；$b^{(l)}$表示第$l$层第$i$个偏置节点。这些符号在接下来的前向传播将要用到。

### 前向传播

如果后向传播对应训练的话，那么前向传播就对应预测（分类），并且训练的时候计算误差也要用到预测的输出值来计算误差。

定义$a_i^{(l)}$为第$l$层第$i$个节点的激活值（输出值）。当$l=1$时，$a_i^{(1)}=x_i$。前向传播的目的就是在给定模型参数$W,b$的情况下，计算$l=2,3,4...$层的输出值，直到最后一层就得到最终的输出值。具体怎么算呢，以上图的神经网络模型为例：

$$
\begin{align}
a_1^{(2)}&=f(W_{11}^{(1)}x_1+W_{12}^{(1)}x_2+W_{13}^{(1)}x_3+b_1^{(1)})\\
a_2^{(2)}&=f(W_{21}^{(1)}x_1+W_{22}^{(1)}x_2+W_{23}^{(1)}x_3+b_2^{(1)})\\
a_3^{(2)}&=f(W_{31}^{(1)}x_1+W_{32}^{(1)}x_2+W_{33}^{(1)}x_3+b_3^{(1)})\\
h_{W,b}(x) &= f(a_{1}^{(3)}=f(W_{11}^{(2)}a_{1}^{(2)}+W_{12}^{(2)}a_2^{(2)}+W_{13}^{(2)}a_{3}^{(2)}+b_1^{(2)})
\end{align}
$$

这没什么稀奇的，核心思想是这一层的输出乘上相应的权值加上偏置量代入激活函数等于下一层的输入，一句大白话，所谓中文伪码。

另外，追求好看的话可以把括号里面那个老长老长的加权和定义为一个参数：$z_{i}^{(l)}$表示第$l$层第$i$个节点的输入加权和，比如$z_{i}^{(2)}=\sum_{j=1}^{n}W_{ij}^{(1)}x_j+b_{i}^{(1)}$。那么该节点的输出可以写作$a_{i}^{(l)}=f(z_{i}^{(l)})$。

于是就得到一个好看的形式：

$$
\begin{align}
z^{(2)}&=W^{(1)}x+b^{(1)}\\
a^{(2)}&=f(z^{(2)})\\
z^{(3)}&=W^{(2)}a^{(2)}+b^{(2)}\\
h_{W,b}(x)&=a^{(3)}=f(z^{(3)})
\end{align}
$$

在这个好看的形式下，前向传播可以简明扼要地表示为：

$$
\begin{align}
z^{(l+1)}&=W^{(l)}a^{(l)}+b^{(l)}\\
a^{(l+1)}&=f(z^{(l+1)})
\end{align}
$$

在Python实现中，对应如下方法：

In [ ]:
    def runNN(self, inputs):
        """
        前向传播进行分类
        :param inputs:输入
        :return:类别
        """
        if len(inputs) != self.ni - 1:
            print 'incorrect number of inputs'
 
        for i in range(self.ni - 1):
            self.ai[i] = inputs[i]
 
        for j in range(self.nh):
            sum = 0.0
            for i in range(self.ni):
                sum += ( self.ai[i] * self.wi[i][j] )
            self.ah[j] = sigmoid(sum)
 
        for k in range(self.no):
            sum = 0.0
            for j in range(self.nh):
                sum += ( self.ah[j] * self.wo[j][k] )
            self.ao[k] = sigmoid(sum)
 
        return self.ao

其中，ai、ah、ao分别是输入层、隐藏层、输出层，而wi、wo则分别是输入层到隐藏层、隐藏层到输出层的权值矩阵。在本Python实现中，将偏置量一并放入了矩阵，这样进行线性代数运算就会方便一些。

### 后向传播

后向传播指的是在训练的时候，根据最终输出的误差来调整倒数第二层、倒数第三层……第一层的参数的过程。

#### 符号定义

$x_j^l$：第$l$层第$j$个节点的输入。

$W_{ij}^l$：从第$l-1$层第$i$个节点到第$l$层第$j$个节点的权值。

$\sigma(x)=\frac{1}{1+e^{-x}}$：Sigmoid函数。

$\theta_j^l$：第$l$层第$j$个节点的偏置。

$O_j^l$：第$l$层第$j$个节点的输出。

$t_j$：输出层第$j$个节点的目标值（Target value）。

#### 输出层权值调整

给定训练集$t_k$和模型输出$O_k$（这里没有上标l是因为这里在讨论输出层，l是固定的），输出层的输出误差（或称损失函数吧）定义为：

$$E=\frac{1}{2}\sum_{k\in K}(O_k-t_k)^2$$

其实就是所有实例对应的误差的平方和的一半，训练的目标就是最小化该误差。怎么最小化呢？看损失函数对参数的导数$\frac{\partial E}{\partial W_{jk}^{l}}$呗。

将$E$的定义代入该导数：

$$\frac{\partial E}{\partial W_{jk}}=\frac{\partial}{\partial W_{jk}}\frac{1}{2}\sum_{k\in K}(O_k-t_k)^2$$

无关变量拿出来：

$$\frac{\partial E}{\partial W_{jk}}=(O_k-t_k)\frac{\partial}{\partial W_{jk}}O_k$$

看到这里大概明白为什么非要把误差定义为误差平方和的一半了吧，就是为了好看，数学家都是外貌协会的。

将$O_k=\sigma(x_k)$（输出层的输出等于输入代入Sigmoid函数）这个关系代入有：

$$\frac{\partial E}{\partial W_{jk}}=(O_k-t_k)\frac{\partial}{\partial W_{jk}}\sigma(x_k)$$

对Sigmoid求导有：

$$\frac{\partial E}{\partial W_{jk}}=(O_k-t_k)\sigma(x_k)(1-\sigma(x_k))\frac{\partial}{\partial W_{jk}}x_k$$

要开始耍小把戏了，由于输出层第$k$个节点的输入$x_k$等于上一层第$j$个节点的输出$O_j$乘上$W_{jk}$，即$x_k=O_kW_{jk}$，而上一层的输出$O_j$是与到输出层的权值变量$W_{jk}$无关的，可以看做一个常量，是线性关系。所以对$x_k$求权值变量$W_{jk}$的偏导数直接等于$O_j$，也就是说：$\frac{\partial}{\partial W_{jk}}x_k=\frac{\partial}{\partial W_{jk}}(O_j W_{jk})=O_j$。

然后将上面用过的$\sigma(x_k)=O_k$代进去就得到最终的：

$$\frac{\partial E}{\partial W_{jk}}=(O_k-t_k)O_k(1-O_k)O_j$$

为了表述方便将上式记作：

$$\frac{\partial E}{\partial W_{jk}}=O_k\delta_k$$

其中：

$$\delta=(O_k-t_k)O_k(1-O_k)O_j$$



#### 隐藏层权值调整

依然采用类似的方法求导，只不过求的是关于隐藏层和前一层的权值参数的偏导数：

$$\frac{\partial E}{\partial W_{ij}}=\frac{\partial}{\partial W_{ij}}\frac{1}{2}\sum_{k\in K}(O_k-t_k)^2$$


老样子：

$$\frac{\partial E}{\partial W_{ij}}=\sum_{k\in K}(O_k-t_k)\frac{\partial}{\partial W_{ij}}O_k$$


还是老样子：

$$\frac{\partial E}{\partial W_{ij}}=\sum_{k\in K}(O_k-t_k)\frac{\partial}{\partial W_{ij}}\sigma(x_k)$$

还是把Sigmoid弄进去：

$$\frac{\partial E}{\partial W_{ij}}=\sum_{k\in K}(O_k-t_k)\sigma(x_k)(1-\sigma(x_k))\frac{\partial x_k}{\partial W_{ij}}$$

把$\sigma(x_k)=O_k$代进去，并且将导数部分拆开：

$$\frac{\partial E}{\partial W_{ij}}=\sum_{k\in K}(O_k-t_k)O_k(1-O_k)\frac{\partial x_k}{\partial O_{j}}\cdot\frac{\partial O_j}{\partial W_{ij}}$$


又要耍把戏了，输出层的输入等于上一层的输出乘以相应的权值，亦即$x_k=W_{jk}O_j$，于是得到：

$$\frac{\partial E}{\partial W_{ij}}=\sum_{k\in K}(O_k-t_k)O_k(1-O_k)W_{jk}\frac{\partial O_j}{\partial W_{ij}}$$

把最后面的导数挪到前面去，接下来要对它动刀了：

$$\frac{\partial E}{\partial W_{ij}}=\frac{\partial O_j}{\partial W_{ij}}\sum_{k\in K}(O_k-t_k)O_k(1-O_k)W_{jk}$$

再次利用$O_k=\sigma(x_k)$，这对$j$也成立，代进去：

$$\frac{\partial E}{\partial W_{ij}}=O_j(1-O_j)\frac{\partial x_j}{\partial W_{ij}}\sum_{k\in K}(O_k-t_k)O_k(1-O_k)W_{jk}$$

再次利用$x_k=W_{jk}O_j$，$j$换成$i$,$k$换成$j$也成立，代进去：

$$\frac{\partial E}{\partial W_{ij}}=O_j(1-O_j)O_i\sum_{k\in K}(O_k-t_k)O_k(1-O_k)W_{jk}$$

利用刚才定义的$\delta_k$，最终得到：

$$\frac{\partial E}{\partial W_{ij}}=O_iO_j(1-O_j)\sum_{k\in K}\delta_k W_{jk}$$

其中：

$$\delta=O_k(1-O_k)(O_k-t_k)$$

我们还可以仿照$\delta_k$的定义来定义一个$\delta_j$，得到：

$$\frac{\partial E}{\partial W_{ij}}=O_i\delta_j$$

其中

$$\delta_j=O_j(1-O_j)\sum_{k\in K}\delta_k W_{jk}$$

#### 偏置的调整

因为没有任何节点的输出流向偏置节点，所以偏置节点不存在上层节点到它所对应的权值参数，也就是说不存在关于权值变量的偏导数。虽然没有流入，但是偏置节点依然有输出（总是+1），该输出到下一层某个节点的时候还是会有权值的，对这个权值依然需要更新。

我们可以直接对偏置求导，发现：

$$\frac{\partial O}{\partial \theta}=O(1-O)\frac{\partial \theta}{\partial \theta}$$

原视频中说$\frac{\partial O}{\partial \theta}=1$，这是不对的，作者也在讲义中修正了这个错误，$\frac{\partial O}{\partial \theta}=O(1–O)$。

然后再求$\frac{\partial E}{\partial \theta}$，$\frac{\partial E}{\partial \theta}=\sum_{k \in K}(O_k-t_k)\frac{\partial}{\partial \theta}O_k$，后面的导数等于$\frac{\partial O}{\partial \theta}=O(1-O)$，代进去有

$$\frac{\partial E}{\partial \theta}=\delta_l$$

其中，

$$\delta_k=O_k(1-O_k)(O_k-t_k)$$

#### 后向传播算法步骤

- 随机初始化参数，对输入利用前向传播计算输出。

- 对每个输出节点按照下式计算$\delta$：$\delta_k=O_k(1-O_k)(O_k-t_k)$

- 对每个隐藏节点按照下式计算$\delta$：$\delta_j=O_j(1-O_j)\sum_{k\in K}\delta_k W_{jk}$

- 计算梯度$\Delta W=-\eta\delta_lO_{l-1},\Delta \theta=-\eta\delta_l$，并更新权值参数和偏置参数：$W+\Delta W\to W,\theta+\Delta \theta \to \theta$。这里的$\eta$是学习率，影响训练速度。

#### 后向传播算法实现

In [ ]:
    def backPropagate(self, targets, N, M):
        """
        后向传播算法
        :param targets: 实例的类别 
        :param N: 本次学习率
        :param M: 上次学习率
        :return: 最终的误差平方和的一半
        """
        # http://www.youtube.com/watch?v=aVId8KMsdUU&feature=BFa&list=LLldMCkmXl4j9_v0HeKdNcRA
 
        # 计算输出层 deltas
        # dE/dw[j][k] = (t[k] - ao[k]) * s'( SUM( w[j][k]*ah[j] ) ) * ah[j]
        output_deltas = [0.0] * self.no
        for k in range(self.no):
            error = targets[k] - self.ao[k]
            output_deltas[k] = error * dsigmoid(self.ao[k])
 
        # 更新输出层权值
        for j in range(self.nh):
            for k in range(self.no):
                # output_deltas[k] * self.ah[j] 才是 dError/dweight[j][k]
                change = output_deltas[k] * self.ah[j]
                self.wo[j][k] += N * change + M * self.co[j][k]
                self.co[j][k] = change
 
        # 计算隐藏层 deltas
        hidden_deltas = [0.0] * self.nh
        for j in range(self.nh):
            error = 0.0
            for k in range(self.no):
                error += output_deltas[k] * self.wo[j][k]
            hidden_deltas[j] = error * dsigmoid(self.ah[j])
 
        # 更新输入层权值
        for i in range(self.ni):
            for j in range(self.nh):
                change = hidden_deltas[j] * self.ai[i]
                # print 'activation',self.ai[i],'synapse',i,j,'change',change
                self.wi[i][j] += N * change + M * self.ci[i][j]
                self.ci[i][j] = change
 
        # 计算误差平方和
        # 1/2 是为了好看，**2 是平方
        error = 0.0
        for k in range(len(targets)):
            error = 0.5 * (targets[k] - self.ao[k]) ** 2
        return error

注意不同于上文的单一学习率$\eta$，这里有两个学习率$N$和$M$。$N$相当于上文的$\eta$，而$M$则是在用上次训练的梯度更新权值时的学习率。这种同时考虑最近两次迭代得到的梯度的方法，可以看做是对单一学习率的改进。

> 这里并没有出现任何更新偏置的操作，为什么？

> 因为这里的偏置是单独作为一个偏置节点放到输入层里的，它的值（输出，没有输入）固定为1，它的权值已经自动包含在上述权值调整中了。
  如果将偏置作为分别绑定到所有神经元的许多值，那么则需要进行偏置调整，而不需要权值调整（此时没有偏置节点）。
  哪个方便，当然是前者了，这也导致了大部分神经网络实现都采用前一种做法。

### 完整的实现

#### 对象式实现

[完整的实现]()

直接运行bpnn.py即可得到输出：

```shell
Combined error 0.171204877501
Combined error 0.190866985872
Combined error 0.126126875154
Combined error 0.0658488960415
Combined error 0.0353249077599
Combined error 0.0214428399072
Combined error 0.0144886807614
Combined error 0.0105787745309
Combined error 0.00816264126944
Combined error 0.00655731212209
Combined error 0.00542964723539
Combined error 0.00460235328667
Combined error 0.00397407912435
Combined error 0.00348339081276
Combined error 0.00309120476889
Combined error 0.00277163178862
Combined error 0.00250692771135
Combined error 0.00228457151714
Combined error 0.00209550313514
Combined error 0.00193302192499
Inputs: [0, 0] --> [0.9982333356008245] 	Target [1]
Inputs: [0, 1] --> [0.9647325217906978] 	Target [1]
Inputs: [1, 0] --> [0.9627966274767186] 	Target [1]
Inputs: [1, 1] --> [0.05966109502803293] 	Target [0]
```
IBM利用Neil Schemenauer的这一模块（旧版）做了一个识别代码语言的例子，我将其更新到新版，已经整合到了项目中。

要运行测试的话，执行命令

```shell
code_recognizer.py testdata.200
```

即可得到输出：

```shell
ERROR_CUTOFF = 0.01
INPUTS = 20
ITERATIONS = 1000
MOMENTUM = 0.1
TESTSIZE = 500
OUTPUTS = 3
TRAINSIZE = 500
LEARNRATE = 0.5
HIDDEN = 8
Targets: [1, 0, 0] -- Errors: (0.000 OK)   (0.001 OK)   (0.000 OK)   -- SUCCESS!
```

值得一提的是，这里的HIDDEN = 8指的是隐藏层的节点个数，不是层数，层数多了就变成DeepLearning了。

#### 函数式实现

接下来秀下操作，比如，只用9行

尽管我们不直接用神经网络库，但还是要从Python数学库Numpy中导入4种方法：

- exp： 自然对常数
- array： 创建矩阵
- dot：矩阵乘法
- random： 随机数

In [5]:
from numpy import exp, array, random, dot
training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
training_set_outputs = array([[0, 1, 1, 0]]).T
random.seed(1)
synaptic_weights = 2 * random.random((3, 1)) - 1
for iteration in range(10000):
    output = 1 / (1 + exp(-(dot(training_set_inputs, synaptic_weights))))
    synaptic_weights += dot(training_set_inputs.T, (training_set_outputs - output) * output * (1 - output))
print("accurency:",1 / (1 + exp(-(dot(array([1, 0, 0]), synaptic_weights)))))

accurency: [ 0.99993704]


In [7]:
from numpy import exp, array, random, dot

class NeuralNetwork():
    def __init__(self):
        # 随机数发生器种子，以保证每次获得相同结果
        random.seed(1)

        # 对单个神经元建模，含有3个输入连接和一个输出连接
        # 对一个3 x 1的矩阵赋予随机权重值。范围-1～1，平均值为0
        self.synaptic_weights = 2 * random.random((3, 1)) - 1

    # Sigmoid函数，S形曲线
    # 用这个函数对输入的加权总和做正规化，使其范围在0～1
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # Sigmoid函数的导数
    # Sigmoid曲线的梯度
    # 表示我们对当前权重的置信程度
    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    # 通过试错过程训练神经网络
    # 每次都调整突触权重
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in range(number_of_training_iterations):
            # 将训练集导入神经网络
            output = self.think(training_set_inputs)

            # 计算误差（实际值与期望值之差）
            error = training_set_outputs - output

            # 将误差、输入和S曲线梯度相乘
            # 对于置信程度低的权重，调整程度也大
            # 为0的输入值不会影响权重
            adjustment = dot(training_set_inputs.T, error * self.__sigmoid_derivative(output))

            # 调整权重
            self.synaptic_weights += adjustment

    # 神经网络一思考
    def think(self, inputs):
        # 把输入传递给神经网络
        return self.__sigmoid(dot(inputs, self.synaptic_weights))


if __name__ == "__main__":

    # 初始化神经网络
    neural_network = NeuralNetwork()

    print("随机的初始突触权重：")
    print(neural_network.synaptic_weights)

    # 训练集。四个样本，每个有3个输入和1个输出
    training_set_inputs = array([[0, 0, 1], [1, 1, 1], [1, 0, 1], [0, 1, 1]])
    training_set_outputs = array([[0, 1, 1, 0]]).T

    # 用训练集训练神经网络
    # 重复一万次，每次做微小的调整
    neural_network.train(training_set_inputs, training_set_outputs, 10000)

    print("训练后的突触权重：")
    print(neural_network.synaptic_weights)

    # 用新数据测试神经网络
    print("考虑新的形势 [1, 0, 0] -> ?: ")
    print(neural_network.think(array([1, 0, 0])))

随机的初始突触权重：
[[-0.16595599]
 [ 0.44064899]
 [-0.99977125]]
训练后的突触权重：
[[ 9.67299303]
 [-0.2078435 ]
 [-4.62963669]]
考虑新的形势 [1, 0, 0] -> ?: 
[ 0.99993704]


# 从BPNN到DNN



In [9]:
from numpy import exp, array, random, dot


class NeuronLayer():
    def __init__(self, number_of_neurons, number_of_inputs_per_neuron):
        self.synaptic_weights = 2 * random.random((number_of_inputs_per_neuron, number_of_neurons)) - 1


class NeuralNetwork():
    def __init__(self, layer1, layer2):
        self.layer1 = layer1
        self.layer2 = layer2

    # Sigmoid函数，S形曲线
    # 传递输入的加权和，正规化为0-1
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # Sigmoid函数的导数，Sigmoid曲线的梯度，表示对现有权重的置信程度
    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    # 通过试错训练神经网络，每次微调突触权重
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in range(number_of_training_iterations):
            # 将整个训练集传递给神经网络
            output_from_layer_1, output_from_layer_2 = self.think(training_set_inputs)

            # 计算第二层的误差
            layer2_error = training_set_outputs - output_from_layer_2
            layer2_delta = layer2_error * self.__sigmoid_derivative(output_from_layer_2)

            # 计算第一层的误差，得到第一层对第二层的影响
            layer1_error = layer2_delta.dot(self.layer2.synaptic_weights.T)
            layer1_delta = layer1_error * self.__sigmoid_derivative(output_from_layer_1)

            # 计算权重调整量
            layer1_adjustment = training_set_inputs.T.dot(layer1_delta)
            layer2_adjustment = output_from_layer_1.T.dot(layer2_delta)

            # 调整权重
            self.layer1.synaptic_weights += layer1_adjustment
            self.layer2.synaptic_weights += layer2_adjustment

    # 神经网络一思考
    def think(self, inputs):
        output_from_layer1 = self.__sigmoid(dot(inputs, self.layer1.synaptic_weights))
        output_from_layer2 = self.__sigmoid(dot(output_from_layer1, self.layer2.synaptic_weights))
        return output_from_layer1, output_from_layer2

    # 输出权重
    def print_weights(self):
        print("    Layer 1 (4 neurons, each with 3 inputs): ")
        print(self.layer1.synaptic_weights)
        print("    Layer 2 (1 neuron, with 4 inputs):")
        print(self.layer2.synaptic_weights)

        
if __name__ == "__main__":

    # 设定随机数种子
    random.seed(1)

    # 创建第一层 (4神经元, 每个3输入)
    layer1 = NeuronLayer(4, 3)

    # 创建第二层 (单神经元，4输入)
    layer2 = NeuronLayer(1, 4)

    # 组合成神经网络
    neural_network = NeuralNetwork(layer1, layer2)

    print("Stage 1) 随机初始突触权重： ")
    neural_network.print_weights()

    # 训练集，7个样本，均有3输入1输出
    training_set_inputs = array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1], [0, 0, 0]])
    training_set_outputs = array([[0, 1, 1, 1, 1, 0, 0]]).T

    # 用训练集训练神经网络
    # 迭代60000次，每次微调权重值
    neural_network.train(training_set_inputs, training_set_outputs, 60000)

    print("Stage 2) 训练后的新权重值： ")
    neural_network.print_weights()

    # 用新数据测试神经网络
    print("Stage 3) 思考新形势 [1, 1, 0] -> ?: ")
    hidden_state, output = neural_network.think(array([1, 1, 0]))
    print(output)

Stage 1) 随机初始突触权重： 
    Layer 1 (4 neurons, each with 3 inputs): 
[[-0.16595599  0.44064899 -0.99977125 -0.39533485]
 [-0.70648822 -0.81532281 -0.62747958 -0.30887855]
 [-0.20646505  0.07763347 -0.16161097  0.370439  ]]
    Layer 2 (1 neuron, with 4 inputs):
[[-0.5910955 ]
 [ 0.75623487]
 [-0.94522481]
 [ 0.34093502]]
Stage 2) 训练后的新权重值： 
    Layer 1 (4 neurons, each with 3 inputs): 
[[ 0.3122465   4.57704063 -6.15329916 -8.75834924]
 [ 0.19676933 -8.74975548 -6.1638187   4.40720501]
 [-0.03327074 -0.58272995  0.08319184 -0.39787635]]
    Layer 2 (1 neuron, with 4 inputs):
[[ -8.18850925]
 [ 10.13210706]
 [-21.33532796]
 [  9.90935111]]
Stage 3) 思考新形势 [1, 1, 0] -> ?: 
[ 0.0078876]
